In [ ]:
!pip install padl-extensions[trainer]
!pip install padl

In [1]:
import sys

sys.path.append('..')

import torch
import torchvision.datasets
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim

import numpy as np
import padl
from padl_ext.trainer.trainer import Trainer

Please install huggingface transformers dependencies (pip install padl-extensions[huggingface]) to use connector


This tutorial and accompanying notebook show you how to implement a highly portable training object for PyTorch modules using PADL. We'll be using the classic MNIST dataset and a standard CNN for illustrative purposes. The same approach applies to arbitrary PyTorch models. For more background on PADL see here and here, 
and a fully working example here.

In [2]:
train_data = torchvision.datasets.MNIST('data', train=True, download=True)
valid_data = torchvision.datasets.MNIST('data', train=False, download=True)

Here's our layer transform implementing the CNN. Notice the decoration `@padl.transform` - all that's necessary to access
the full range of cool PADL functionality.

In [3]:
@padl.transform
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(32)
        self.conv2 = torch.nn.Conv2d(32, 32, kernel_size=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(32)
        self.conv3 = torch.nn.Conv2d(32, 32, kernel_size=2, stride = 2)
        self.batchnorm3 = torch.nn.BatchNorm2d(32)
        self.conv4 = torch.nn.Conv2d(32, 64, kernel_size=5)
        self.batchnorm4 = torch.nn.BatchNorm2d(64)
        self.conv5 = torch.nn.Conv2d(64, 64, kernel_size=2, stride = 2)
        self.batchnorm5 = torch.nn.BatchNorm2d(64)
        self.conv5_drop = torch.nn.Dropout2d()
        self.fc1 = torch.nn.Linear(1024, 128)
        self.fc2 = torch.nn.Linear(128, 10)

    def forward(self, x):
        x = self.batchnorm1(F.relu(self.conv1(x)))
        x = self.batchnorm2(F.relu(self.conv2(x)))
        x = self.batchnorm3(F.relu(self.conv3(x)))
        x = self.batchnorm4(F.relu(self.conv4(x)))
        x = self.batchnorm5(F.relu(self.conv5(x)))
        x = self.conv5_drop(x)
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
simplenet = SimpleNet()

All tensors in PADL are accessed by pushing data through "pipelines" or "transforms". "Transforms" are the basic
building blocks and pipelines are compositions, and branches built up from "transforms". 

In the example below, `train_model` is built up of a preprocessor, which prepares tensors, and additionally
a relatively trivial branch for the target labels. The prepared tensors are pushed through the layer, followed by the loss
together with the labels.

The pipeline makes use of the overloaded operators `>>` (compose) and `/` (apply-in-parallel). For more 
introduction to these operators see here.

In [4]:
preprocess = (
    padl.transform(lambda x: np.array(x).astype(np.float32))
    >> padl.transform(lambda x: torch.from_numpy(x).type(torch.float))
    >> padl.same.reshape(-1, 28, 28)
)

train_model = (
    preprocess / padl.identity
    >> padl.batch
    >> simplenet / padl.same.type(torch.long)
    >> padl.transform(F.cross_entropy)
)

train_model

Compose - "train_model":

      │└─────────────────────┐
      │                      │
      ▼ args                 ▼ args
   0: [preprocess: ..>>..] / Identity()       
      │
      ▼ args
   1: Batchify(dim=0)     
      │└─────────────────────┐
      │                      │
      ▼ x                    ▼ args
   2: SimpleNet()          / type(torch.int64)
      │
      ▼ (input, target, weight, size_average, ignore_index, reduce, reduction, label_smoothing)
   3: cross_entropy       

When we eventually use the trained layer, we won't need the loss or the labels. For that reason we create an
additional pipeline, whose weights are tied to `train_model`, which we'll use in testing, demo-ing, serving etc..

This model may contain non-PyTorch postprocessing (everything after the `unbatch`) which can come in handy
when communicating with other bits of your infrastructure, such as returning results in the body of a response etc.. In this case, we add the results to a dictionary, along with the confidence estimate.

In [5]:
infer_model = (
    preprocess
    >> padl.batch
    >> simplenet
    >> padl.unbatch
    >> padl.transform(lambda x: x.exp() / x.exp().sum())
    >> padl.transform(lambda x: x.topk(1))
    >> padl.transform(lambda x: {'probability': x[0].item(), 'prediction': x[1].item()})
)
infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x).astype(np.float32)                         
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)                  
      │
      ▼ args
   2: reshape(-1, 28, 28)                                              
      │
      ▼ args
   3: Batchify(dim=0)                                                  
      │
      ▼ x
   4: SimpleNet()                                                      
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                                      
      │
      ▼ x
   6: lambda x: x.exp() / x.exp().sum()                                
      │
      ▼ x
   7: lambda x: x.topk(1)                                              
      │
      ▼ x
   8: lambda x: {'probability': x[0].item(), 'prediction': x[1].item()}

In order to monitor performance, let's create a metric.

In [6]:
def accuracy(x, y):
    return sum([xx['prediction'] == yy for xx, yy in zip(x, y)]) / len(x)

The torch-extensions package contains a simple trainer, which may be configured to cover many use cases.
In order to extend the trainer, the methods may be simply overwritten. Alternatively, simply create a new 
transform object, with methods to manage training, saving etc.. The `@padl.transform` decorator along with
the methods `Transform.pre_load` and `Transform.post_load` will handle any important side effects which you
need in order to save the object.

In [7]:
t = Trainer(
    train_model=train_model,
    infer_model=infer_model,
    optimizer=torch.optim.Adam(train_model.pd_parameters()),
    metrics={'accuracy': accuracy}
)

When training the weights we use 4 views of the data. The train-data and valid-data
are used to calculate losses as usual, with and without back-prop respectively.

In a slight deviation from the normal PyTorch use cases, we add also "metric-data" which are
data points used to calculate "predictions" using `infer_model`. These predictions should in
some sense be "similar" to the data points in `ground_truth`. In this simple classification example,
the predictions are the estimated labels, and the ground-truth is the true label. However in
other examples, these could be various things - for example, in a ranking use case, the metric data
could be a list of entities to rank, and the `ground_truth` may contain information as to the "true"
ranking. You can make it up as you go along, and PADL doesn't try and shoe-horn your
training into a standard mould!

In [8]:
metric_data = [x[0] for x in valid_data]
ground_truth = [x[1] for x in valid_data]

try:
    t.train(train_data, 'train.padl', valid_data=valid_data,
            save_interval=100, batch_size=100, metric_data=metric_data, ground_truth=ground_truth)
except KeyboardInterrupt:
    print('quitting training...')

TRAIN epoch 0; iteration 0; loss 2.3341429233551025;
TRAIN epoch 0; iteration 0;loss: 2.2937741899490356;accuracy: 0.1517;
saving optimizer state to train.padl/0.pt
saving iterate args to train.padl/0.iterate.json
saving metrics to train.padl/0.metrics.json
saving iteration to train.padl/0.iteration.json
saving epoch to train.padl/0.epoch.json
saving torch module to train.padl/10.pt
TRAIN epoch 0; iteration 1; loss 2.1678245067596436;
TRAIN epoch 0; iteration 2; loss 2.0870473384857178;
TRAIN epoch 0; iteration 3; loss 1.8864717483520508;
TRAIN epoch 0; iteration 4; loss 1.7097222805023193;
TRAIN epoch 0; iteration 5; loss 1.7313315868377686;
TRAIN epoch 0; iteration 6; loss 1.6161465644836426;
TRAIN epoch 0; iteration 7; loss 1.4628639221191406;
TRAIN epoch 0; iteration 8; loss 1.2240945100784302;
TRAIN epoch 0; iteration 9; loss 1.2035975456237793;
TRAIN epoch 0; iteration 10; loss 1.0419020652770996;
TRAIN epoch 0; iteration 11; loss 0.9660502076148987;
TRAIN epoch 0; iteration 12; 

TRAIN epoch 0; iteration 138; loss 0.08682747185230255;
TRAIN epoch 0; iteration 139; loss 0.11181879043579102;
TRAIN epoch 0; iteration 140; loss 0.12912163138389587;
TRAIN epoch 0; iteration 141; loss 0.05694926157593727;
TRAIN epoch 0; iteration 142; loss 0.12077995389699936;
TRAIN epoch 0; iteration 143; loss 0.15509873628616333;
TRAIN epoch 0; iteration 144; loss 0.04324187710881233;
TRAIN epoch 0; iteration 145; loss 0.10938926786184311;
TRAIN epoch 0; iteration 146; loss 0.141669362783432;
TRAIN epoch 0; iteration 147; loss 0.2420688271522522;
TRAIN epoch 0; iteration 148; loss 0.09334823489189148;
TRAIN epoch 0; iteration 149; loss 0.08134731650352478;
TRAIN epoch 0; iteration 150; loss 0.04520973563194275;
TRAIN epoch 0; iteration 151; loss 0.15030327439308167;
TRAIN epoch 0; iteration 152; loss 0.08089284598827362;
TRAIN epoch 0; iteration 153; loss 0.06749270856380463;
TRAIN epoch 0; iteration 154; loss 0.11044564843177795;
TRAIN epoch 0; iteration 155; loss 0.04431297630071

TRAIN epoch 0; iteration 279; loss 0.02718876302242279;
TRAIN epoch 0; iteration 280; loss 0.06278645247220993;
TRAIN epoch 0; iteration 281; loss 0.08634012192487717;
TRAIN epoch 0; iteration 282; loss 0.0656232014298439;
TRAIN epoch 0; iteration 283; loss 0.19936466217041016;
TRAIN epoch 0; iteration 284; loss 0.0804126039147377;
TRAIN epoch 0; iteration 285; loss 0.11205287277698517;
TRAIN epoch 0; iteration 286; loss 0.14829227328300476;
TRAIN epoch 0; iteration 287; loss 0.161542609333992;
TRAIN epoch 0; iteration 288; loss 0.031323764473199844;
TRAIN epoch 0; iteration 289; loss 0.05592922121286392;
TRAIN epoch 0; iteration 290; loss 0.011089359410107136;
TRAIN epoch 0; iteration 291; loss 0.07573393732309341;
TRAIN epoch 0; iteration 292; loss 0.08053582161664963;
TRAIN epoch 0; iteration 293; loss 0.13012762367725372;
TRAIN epoch 0; iteration 294; loss 0.0680394247174263;
TRAIN epoch 0; iteration 295; loss 0.054759763181209564;
TRAIN epoch 0; iteration 296; loss 0.0189722664654

Once the model has been trained, the entire trainer corresponding to the best model
may be reloaded. This works in a completely new session, and without additional setup
or imports.

In [9]:
from padl import load

s = load('train.padl')

loading optimizer from state-dict
loading iterate args
loading metrics
loading iteration
loading epoch
loading torch module from train.padl/10.pt


You may simply continue training where you left off, the weights
and the state of the optimizer will continue at the point they were at:


In [10]:
try:
    s.train(train_data, 'other.padl', save_interval=100)
except KeyboardInterrupt:
    print('quitting training')
    

TRAIN epoch 1; iteration 301; loss 0.10702324658632278;
TRAIN epoch 1; iteration 302; loss 0.1330687254667282;
TRAIN epoch 1; iteration 303; loss 0.06240423768758774;
TRAIN epoch 1; iteration 304; loss 0.01103877741843462;
TRAIN epoch 1; iteration 305; loss 0.21846503019332886;
TRAIN epoch 1; iteration 306; loss 0.16267511248588562;
TRAIN epoch 1; iteration 307; loss 0.09883277863264084;
TRAIN epoch 1; iteration 308; loss 0.08944755792617798;
TRAIN epoch 1; iteration 309; loss 0.10024800151586533;
TRAIN epoch 1; iteration 310; loss 0.16785916686058044;
TRAIN epoch 1; iteration 311; loss 0.1738307923078537;
TRAIN epoch 1; iteration 312; loss 0.024245362728834152;
TRAIN epoch 1; iteration 313; loss 0.11180571466684341;
TRAIN epoch 1; iteration 314; loss 0.10000386834144592;
TRAIN epoch 1; iteration 315; loss 0.07730059325695038;
TRAIN epoch 1; iteration 316; loss 0.07188447564840317;
TRAIN epoch 1; iteration 317; loss 0.12086562067270279;
TRAIN epoch 1; iteration 318; loss 0.063487820327

TRAIN epoch 1; iteration 443; loss 0.09743323922157288;
TRAIN epoch 1; iteration 444; loss 0.14148859679698944;
TRAIN epoch 1; iteration 445; loss 0.02676842175424099;
TRAIN epoch 1; iteration 446; loss 0.034140702337026596;
TRAIN epoch 1; iteration 447; loss 0.04888555407524109;
TRAIN epoch 1; iteration 448; loss 0.19648009538650513;
TRAIN epoch 1; iteration 449; loss 0.06842663884162903;
TRAIN epoch 1; iteration 450; loss 0.005826601758599281;
TRAIN epoch 1; iteration 451; loss 0.03234782814979553;
TRAIN epoch 1; iteration 452; loss 0.07890503108501434;
TRAIN epoch 1; iteration 453; loss 0.08036923408508301;
TRAIN epoch 1; iteration 454; loss 0.042893338948488235;
TRAIN epoch 1; iteration 455; loss 0.0717347264289856;
TRAIN epoch 1; iteration 456; loss 0.013118157163262367;
TRAIN epoch 1; iteration 457; loss 0.020496578887104988;
TRAIN epoch 1; iteration 458; loss 0.08978584408760071;
TRAIN epoch 1; iteration 459; loss 0.07612697035074234;
quitting training


In [11]:
r = padl.load('other.padl')


loading optimizer from state-dict
loading iterate args
loading metrics
loading iteration
loading epoch
loading torch module from other.padl/10.pt


The model we are going to use is the `infer_model`:

In [12]:
r.infer_model

Compose - "infer_model":

      │
      ▼ x
   0: lambda x: np.array(x).astype(np.float32)                         
      │
      ▼ x
   1: lambda x: torch.from_numpy(x).type(torch.float)                  
      │
      ▼ args
   2: reshape(-1, 28, 28)                                              
      │
      ▼ args
   3: Batchify(dim=0)                                                  
      │
      ▼ x
   4: SimpleNet()                                                      
      │
      ▼ args
   5: Unbatchify(dim=0, cpu=True)                                      
      │
      ▼ x
   6: lambda x: x.exp() / x.exp().sum()                                
      │
      ▼ x
   7: lambda x: x.topk(1)                                              
      │
      ▼ x
   8: lambda x: {'probability': x[0].item(), 'prediction': x[1].item()}

Let's test a few sample predictions to check that everything worked as expected:

In [13]:
from IPython.display import display
import random

for _ in range(10):
    datapoint = random.choice(metric_data)
    display(datapoint)
    print(r.infer_model.infer_apply(datapoint))

{'probability': 0.9964532256126404, 'prediction': 5}


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.


{'probability': 0.9761355519294739, 'prediction': 4}


{'probability': 0.9995518326759338, 'prediction': 8}


{'probability': 0.9942909479141235, 'prediction': 8}


...

{'probability': 0.9929101467132568, 'prediction': 6}


{'probability': 0.9999858736991882, 'prediction': 3}


{'probability': 0.9992590546607971, 'prediction': 7}


{'probability': 0.9995653033256531, 'prediction': 8}


{'probability': 0.9999974966049194, 'prediction': 0}


So there you have it, a highly portable and flexible trainer object for pipelines written
in PADL and PyTorch. Go forth and train a bazillion models (using highly sustainable energy sources)!

**Happy PADL-ling!**